In [12]:
ufs = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']


from uuid import uuid4
from random import choice, random, randrange, randint
from datetime import datetime, timedelta

create_user= {
    'id': lambda: str(uuid4()),
    'uf': lambda : choice(ufs)
}

users = [{k:v() for k,v in create_user.items()} for i in range(10)]

create_music ={
    'id':lambda: str(uuid4()),
    'artist_uf': lambda : choice(ufs)
}

musics = [{k:v() for k,v in create_music.items()} for i in range(50)]

create_stream={
    'id':lambda: str(uuid4()),
    'user': lambda : choice(users)['id'],
    'music':lambda : choice(musics)['id'],
    'datetime': lambda : (datetime.now()- timedelta(seconds= randrange(60*60*24))).strftime('%Y-%m-%dT%H:%M%S')
}

streams=  [{k:v() for k,v in create_stream.items()} for i in range(500)]

create_sequence = {
    'id':lambda: str(uuid4()),
    'user': lambda : choice(users)['id'],
    'previous': lambda : choice(musics)['id'],
    'next': lambda : choice(musics)['id'],
    'percentage_lisned': lambda: random()*100,
}

sequences =  [{k:v() for k,v in create_sequence.items()} for i in range(499)]

create_playlist = {
    'id':lambda: str(uuid4()),
    'user': lambda : choice(users)['id'],
    'music1': lambda : choice(musics)['id'],
    'music2': lambda : choice(musics)['id'],
    'distance': lambda: randint(0,20),
}

playlists =  [{k:v() for k,v in create_playlist.items()} for i in range(100)]


create_model = {
    "model_type": lambda: "random_forest",
              "estimators":lambda: randint(300,500),
                "accuracy" :lambda:randrange(30,90),
                "precision" :lambda: random(),
                "mcc" :  lambda:random(),
                "datetime":lambda: (datetime.now()- timedelta(days=  random()*7)).strftime('%Y-%m-%dT%H:%M:%S'),
               "id":lambda: str(uuid4())
              }
     
models = [{k:v() for k,v in create_model.items()} for i in range(8)]

In [13]:
import azure.cosmos.cosmos_client as cosmos_client


def update_data(config, data):


    # Cria um cliente do Cosmos DB
    client = cosmos_client.CosmosClient(url=config['ENDPOINT'],credential= {'masterKey': config['PRIMARYKEY']})

    # Cria um novo container no banco de dados, caso ele ainda não exista
    database = client.get_database_client(config['DATABASE'])
    container = database.get_container_client(config['CONTAINER'])

    # Insere cada dicionário da lista como uma linha no banco de dados
    for record in data:
        container.upsert_item(body=record)


In [14]:
config_musics = {
    'ENDPOINT': 'https://fiap-normal-database.documents.azure.com:443/',
    'PRIMARYKEY': 'FpUhmIEBc4hSW2rLsm1xUkrdhdHpPhLGj1uRWFMx2BHzBrcJPr3oRbSDdIFFFQKqZGk4Nloclq2SACDbYIk3BQ==',
    'DATABASE': 'musics',
    'CONTAINER': 'users'
}
update_data(config_musics|{'CONTAINER':'users'}, users)
update_data(config_musics|{'CONTAINER':'songs'}, musics)
update_data(config_musics|{'CONTAINER':'listens'}, streams)
update_data(config_musics|{'CONTAINER':'models'}, models)


In [20]:

config_musics = {
    'ENDPOINT': 'https://fiapgraphdatabase.documents.azure.com:443/',
    'PRIMARYKEY': 'u7c8yDpUBourU636HhHU3WnJJhMz0mOVk1GzKf6fqfpEsudzgUegTZ3yby6Arbzq2OGdKzcwENm4ACDb2jlygw==',
    'DATABASE': 'music_correlations'
}
update_data(config_musics|{'CONTAINER':'playlists'}, playlists)
update_data(config_musics|{'CONTAINER':'sequences'}, sequences)


In [28]:
from requests import post

response = post('http://localhost:7071/api/users', json= users)
print(response.json())
response = post('http://localhost:7071/api/songs', json= musics)
print(response.json())
response = post('http://localhost:7071/api/listens', json= streams)
print(response.json())
response = post('http://localhost:7071/api/playlists', json= playlists)
print(response.json())
response = post('http://localhost:7071/api/sequences', json= sequences)
print(response.json())

{'success': True, 'message': 'Sucesso ao subir 10 registros para o banco de users'}
{'success': True, 'message': 'Sucesso ao subir 50 registros para o banco de songs'}
{'success': True, 'message': 'Sucesso ao subir 500 registros para o banco de listens'}
{'success': True, 'message': 'Sucesso ao subir 100 registros para o banco de playlists'}
{'success': True, 'message': 'Sucesso ao subir 499 registros para o banco de sequences'}


In [25]:
from requests import get

response = get('http://localhost:7071/api/users')
print(str(len(response.json()['result'])) + ' users')
response = get('http://localhost:7071/api/songs')
print(str(len(response.json()['result'])) + ' songs')
response = get('http://localhost:7071/api/playlists')
print(str(len(response.json()['result'])) + ' playlists')
response = get('http://localhost:7071/api/sequences')
print(str(len(response.json()['result'])) + ' sequences')
response = get('http://localhost:7071/api/listens')
print(str(len(response.json()['result'])) + ' listens')

30 users
100 songs
100 playlists
499 sequences
500 listens


In [6]:
randrange(0,0.90)

C:\Users\netoc\AppData\Local\Temp\ipykernel_6140\4128215132.py:1: DeprecationWarning: randrange() will raise TypeError in the future
  randrange(0,0.90)


ValueError: non-integer stop for randrange()

In [5]:
randrange(30,90)

53

In [17]:
import json

json.dumps(streams[0])

'{"id": "e950a501-e46a-4eed-88be-5967c440967f", "user": "5e3b7511-4336-453e-8199-a4ec14956742", "music": "bce99319-a162-4530-8a8c-6a13fbc6fde7", "datetime": "2023-03-23T11:2206"}'

In [6]:

from uuid import uuid4
from random import choice, random, randrange, randint
from datetime import datetime, timedelta
import requests
from io import StringIO
import pandas as pd
with requests.get('https://fiapfunctionapp.azurewebsites.net/api/songs') as response:
    _data = StringIO(response.content.decode('utf-8'))



In [8]:
df = pd.read_csv(_data)

EmptyDataError: No columns to parse from file

In [13]:
musics_id = list(df[df.id.apply(lambda s: '-' not in s)].id)

In [17]:
create_stream={
    'id':lambda: str(uuid4()),
    'user': lambda : "5e3b7511-4336-453e-8199-a4ec14956742",
    'music':lambda : choice(musics_id),
    'datetime': lambda : (datetime.now()- timedelta(seconds= randrange(60*60*24*30))).strftime('%Y-%m-%dT%H:%M:%S')
}

streams=  [{k:v() for k,v in create_stream.items()} for i in range(500)]

In [19]:
requests.post('https://fiapfunctionapp.azurewebsites.net/api/listens',json=streams)

<Response [200]>